In [1]:
from os.path import basename, exists

import warnings
warnings.filterwarnings('ignore')
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")

In [2]:
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/first.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download(
    "https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz"
)

In [3]:
import numpy as np
import statsmodels.formula.api as smf
import random
import pandas as pd
import thinkstats2
import thinkplot

__1.__ Suppose one of your co-workers is expecting a baby and you
are participating in an oﬃce pool to predict the date of birth. Assuming that
bets are placed during the 30th week of pregnancy, what variables could you
use to make the best prediction? You should limit yourself to variables that
are known before the birth, and likely to be available to the people in the
pool.

In [5]:
#bring in the data 
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]
live.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,laborfor_i,religion_i,metro_i,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,9.1250
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,7.0000
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,0,0,0,7226.301740,8567.549110,12999.542264,2,12,NaN,6.1875


In [6]:
#Create patsy function to check the good fit
import patsy
    
def GoMining(df):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = 'prglngth ~ ' + name
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError, patsy.PatsyError) as e:
            continue
        
        variables.append((results.rsquared,results.pvalues[name], name))

    return variables

In [7]:
variables = GoMining(live)

In [8]:
# Sort by fit
outvardf = pd.DataFrame (variables)
outvardf.columns = ['rsquared','pvalues','varname']
variableR = outvardf.sort_values(by=['rsquared'],ascending=False)
print(variableR.to_string())

         rsquared        pvalues          varname
39   1.000000e+00   0.000000e+00         prglngth
8    8.062434e-01   0.000000e+00          wksgest
107  1.244574e-01  1.019547e-255      totalwgt_lb
12   1.197731e-01  9.308211e-247      birthwgt_lb
49   1.037254e-01  1.622321e-213             lbw1
9    9.562432e-02  3.821917e-196          mosgest
75   2.205378e-02   5.528283e-45       prglngth_i
3    4.577566e-03   1.747263e-10         nbrnaliv
18   2.452025e-03   1.040661e-05         anynurse
50   2.369184e-03   1.636460e-05         bfeedwks
2    2.249389e-03   7.742373e-06         pregend1
22   2.043142e-03   2.025960e-05         cmlastlb
83   1.968159e-03   2.876961e-05       fmarcon5_i
27   1.891753e-03   2.380793e-04         evuseint
6    1.657132e-03   1.240548e-04       gestasun_m
106  1.322368e-03   6.076531e-04             sest
17   1.309107e-03   1.222857e-03       matchfound
24   1.282862e-03   7.340248e-04         cmlstprg
40   1.237274e-03   9.132598e-04         birthord


In [9]:
# from above list the variables that suits the question of being aware of before birth are nbrnaliv, birthord, poverty, pregordr 
model = smf.ols('prglngth ~  nbrnaliv>1 + birthord + poverty', data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     32.77
Date:                Wed, 12 Feb 2025   Prob (F-statistic):           4.66e-21
Time:                        00:23:11   Log-Likelihood:                -18250.
No. Observations:                8884   AIC:                         3.651e+04
Df Residuals:                    8880   BIC:                         3.654e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               38.9164      0.055    710.990      0.000      38.809      39.024
nbrnaliv > 1[T.True]    -1.4794      0.165     -8.987      0.000      -1.802      -1.157
birthord                -0.0459      0.020     -2.328      0.020      -0.085      -0.007
poverty                  0.0003      0.000      2.312      0.021    4.96e-05       0.001
==============================================================================
Omnibus:                     1583.184   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6108.873
Skew:                          -0.851   Prob(JB):                         0.00
Kurtosis:                       6.689   Cond. No.                     2.09e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.09e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

_Based on above results the intercept is __38.92__ weeks which is pretty normal length of pragnency. So delivery should happen around that time period. All three variables are having PValues below 0.05 which means, the values are not by chance and are statistically significant._

__2.__ If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [18]:
#Remove missings
join['numbabes'] = join.numbabes.replace([97], np.nan)
join.head()

,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,multbrth,...,basewgt_r,adj_mod_basewgt_r,finalwgt_r,secu_r,sest_r,cmintvw_r,cmlstyr,screentime,intvlngth,boy
0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,3410.389399,3869.349602,6448.271112,2,9,1231,1219,19:56:43,67.563833,1
1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,3410.389399,3869.349602,6448.271112,2,9,1231,1219,19:56:43,67.563833,0
2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,5.0,...,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167,1
3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167,0
4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,NaN,...,7226.301740,8567.549110,12999.542264,2,12,1231,1219,14:54:03,106.018167,0


In [19]:
# Apply poissions regression model on variables
model = smf.poisson('numbabes ~ age_r + C(race) + totincr + educat', data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.687055
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8878
Method:                           MLE   Df Model:                            5
Date:                Wed, 12 Feb 2025   Pseudo R-squ.:                 0.03109
Time:                        00:23:24   Log-Likelihood:                -14988.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.106e-205
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.0842      0.045     23.995      0.000       0.996       1.173
C(race)[T.2]    -0.1398      0.015     -9.464      0.000      -0.169      -0.111
C(race)[T.3]    -0.0914      0.025     -3.717      0.000      -0.140      -0.043
age_r            0.0208      0.001     20.474      0.000       0.019       0.023
totincr         -0.0179      0.002     -9.442      0.000      -0.022      -0.014
educat          -0.0443      0.003    -15.139      0.000      -0.050      -0.039
================================================================================
"""

In [20]:
# Predict the results
columns = ['age_r', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 1, 14, 16]], columns=columns)
results.predict(new)

0    2.342182
dtype: float64

The answer is 2.34, which is more inclined between 2 or 3 childrens at the age of 35. 

__3.__ If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called `mnlogit`. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called `rmarital`.

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [39]:
# using the similar model as used above
join['age2'] = join.age_r**2
# Apply mnlogit regression model on variables
model = smf.mnlogit('rmarital ~ age_r + age2 + C(race) + totincr + educat', data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.084053
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8849
Method:                           MLE   Df Model:                           30
Date:                Wed, 12 Feb 2025   Pseudo R-squ.:                  0.1682
Time:                        00:34:56   Log-Likelihood:                -9630.7
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        9.0156      0.805     11.199      0.000       7.438      10.593
C(race)[T.2]    -0.9237      0.089    -10.418      0.000      -1.097      -0.750
C(race)[T.3]    -0.6179      0.136     -4.536      0.000      -0.885      -0.351
age_r           -0.3635      0.051     -7.150      0.000      -0.463      -0.264
age2             0.0048      0.001      6.103      0.000       0.003       0.006
totincr         -0.1310      0.012    -11.337      0.000      -0.154      -0.108
educat          -0.1953      0.019    -10.424      0.000      -0.232      -0.159
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.9570      3.020      0.979      0.328      -2.963       8.877
C(race)[T.2]    -0.4411      0.237     -1.863      0.062      -0.905       0.023
C(race)[T.3]     0.0591      0.336      0.176      0.860      -0.600       0.718
age_r           -0.3177      0.177     -1.798      0.072      -0.664       0.029
age2             0.0064      0.003      2.528      0.011       0.001       0.011
totincr         -0.3258      0.032    -10.175      0.000      -0.389      -0.263
educat          -0.0991      0.048     -2.050      0.040      -0.194      -0.004
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -3.5238      1.205     -2.924      0.003      -5.886      -1.162
C(race)[T.2]    -0.3213      0.093     -3.445      0.001      -0.504      -0.139
C(race)[T.3]    -0.7706      0.171     -4.509      0.000      -1.106      -0.436
age_r            0.1155      0.071      1.626      0.104      -0.024       0.255
age2            -0.0007      0.001     -0.701      0.483      -0.003       0.001
totincr         -0.2276      0.012    -19.621      0.000      -0.250      -0.205
educat           0.0667      0.017      3.995      0.000       0.034       0.099
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.8963      1.305     -2.220      0.026      -5.453      -0.339
C(race)[T.2]    -1.0407      0.104    -10.038      0.000      -1.244      -0.837
C(race)[T.3]    -0.5661      0.156     -3.635      0.000      -0.871      -0.261
age_r            0.2411      0.079      3.038      0.002       0.086       0.397
age2            -0.0035      0.001     -2.977      0.003      -0.006      -0.001
totincr         -0.2932      0.015    -20.159      0.000      -0.322      -0.265
educat          -0.0174      0.021     -0.813      0.416      -0.059       0.025
-

In [41]:
#Predict the results
columns = ['age_r', 'age2', 'race', 'totincr', 'educat']
df = pd.DataFrame([[25, 25**2, 2, 11, 12]], columns=columns)
results.predict(df)

,0,1,2,3,4,5
0,0.750028,0.126397,0.001564,0.033403,0.021485,0.067122


Based on the results there are 75% chances that she is married. 13% indicate in a relationship or not married 